In [4]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

#### DATA

In [5]:
data = pd.read_csv("/home/namhoai/WorkingSpace/train.csv")


linkcheck = "https://www.kaggle.com/code/wwsalmon/simple-mnist-nn-from-scratch-numpy-no-tf-keras/notebook"

In [6]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 
# nn1
data_dev = data[0:1000]
Y_dev = data_dev[:,0]
X_dev = data_dev[:,1:]
X_dev = X_dev / 255.0

data_train = data[1000:m]
Y_train = data_train[:, 0]
X_train = data_train[:, 1:]
X_train = X_train / 255.0

#### HYPER

In [7]:
input_dim = 784
output_dim = 10
hidden_dim = 10
learning_rate = 0.1
epochs = 500

In [8]:
w1 = np.random.rand(input_dim,hidden_dim) - 0.5
b1 = np.random.rand(1,hidden_dim) - 0.5
w2 = np.random.rand(hidden_dim, output_dim) - 0.5
b2 = np.random.rand(1,output_dim) - 0.5
NN = {"w1": w1, "b1": b1, "w2": w2, "b2": b2}

#### DEFINITION

In [9]:
def softmax(Z):
    exp_scores = np.exp(Z)
    softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return softmax

def relu(x):
    return np.maximum(0, x)


def get_predictions(A2):
    return np.argmax(A2, 0)


def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def get_item(NN):
    w1 = NN["w1"]
    b1 = NN["b1"]
    w2 = NN["w2"]
    b2 = NN["b2"]
    return w1, b1, w2, b2

#### FOR AND BACK

In [10]:
def forward(x, NN):
    W1, b1, W2, b2 = get_item(NN)
    z1 = x.dot(W1) + b1
    a1 = relu(z1)
    z2 = a1.dot(W2) + b2
    a2 = softmax(z2)
    return a1, a2

In [11]:
def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y
    return one_hot_Y

In [12]:
def backward(x, y, NN, learning_rate):

    W1, b1, W2, b2 = get_item(NN)
    a1, a2 = forward(x, NN)

    delta3 = a2 - one_hot(y)

    dW2 = (1 / m) * np.dot(a1.T, delta3)
    db2 = (1 / m) * np.sum(delta3, axis=0, keepdims=True)

    delta2 = np.dot(delta3, W2.T) * ReLU_deriv(a1)
    dW1 = (1 / m) * np.dot(x.T, delta2)
    db1 = (1 / m) * np.sum(delta2, axis=0, keepdims=True)

    # update
    W1 += -learning_rate * dW1
    b1 += -learning_rate * db1
    W2 += -learning_rate * dW2
    b2 += -learning_rate * db2
    NN["w1"] = W1
    NN["b1"] = b1
    NN["w2"] = W2
    NN["b2"] = b2

    return a2

#### TRAIN

In [13]:
for i in range(epochs):

    a2 = backward(X_train, Y_train, NN, learning_rate)
    if i % 50 == 0:
        print("Iteration: ", i)
        predictions = get_predictions(a2.T)
        print(get_accuracy(predictions, Y_train.T))
    

Iteration:  0
[7 7 3 ... 8 8 8] [7 7 2 ... 0 3 7]
0.10504878048780487


Iteration:  50
[1 7 2 ... 8 8 7] [7 7 2 ... 0 3 7]
0.4637317073170732
Iteration:  100
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.6207317073170732
Iteration:  150
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.6915121951219512
Iteration:  200
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.7350487804878049
Iteration:  250
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.7677073170731707
Iteration:  300
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.7915609756097561
Iteration:  350
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.8086585365853659
Iteration:  400
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.8213170731707317
Iteration:  450
[7 7 2 ... 8 3 7] [7 7 2 ... 0 3 7]
0.8315121951219512


#### TEST

In [14]:
def make_predictions(X,NN):
    _, A2 =  forward(X,NN)
    predictions = get_predictions(A2.T)
    return predictions

In [15]:

dev_predictions = make_predictions(X_dev,NN)
get_accuracy(dev_predictions, Y_dev.T)

[3 9 4 0 6 9 9 2 1 1 0 3 5 8 2 6 9 1 3 0 3 8 9 3 5 5 9 2 1 2 7 3 3 9 2 9 0
 2 8 8 7 5 3 8 9 4 1 5 7 1 1 8 8 8 6 5 4 5 1 0 3 5 1 0 6 4 6 8 8 6 5 0 1 5
 6 5 6 8 7 8 1 2 6 9 3 1 1 9 7 6 3 7 6 8 4 3 5 7 3 8 9 7 1 7 0 5 5 2 7 6 0
 8 5 4 9 0 0 3 5 6 0 7 7 2 4 2 4 5 9 3 4 9 0 6 0 5 5 6 1 5 3 5 0 6 7 3 5 9
 2 6 8 6 7 2 5 6 8 8 0 7 4 6 2 8 6 5 6 3 4 5 2 0 7 5 3 2 0 9 2 8 7 4 4 7 9
 1 6 0 1 1 0 3 1 9 0 3 1 7 0 5 1 6 1 6 3 2 2 2 1 3 2 0 1 8 3 0 4 2 9 1 4 6
 0 2 4 3 4 9 1 1 5 3 1 0 4 9 1 3 9 0 7 9 1 7 6 1 9 0 6 2 2 7 3 5 8 0 6 2 9
 4 8 0 2 7 0 4 6 6 0 5 1 5 6 5 7 7 7 4 1 7 8 8 1 2 8 9 0 6 7 6 8 7 8 3 0 0
 7 8 3 3 9 9 6 7 3 0 7 7 0 6 2 0 9 4 8 0 0 8 5 6 9 9 8 3 3 4 0 4 9 4 4 7 6
 3 6 6 0 2 1 3 4 5 1 8 6 6 2 1 1 9 8 4 3 3 7 3 3 0 0 8 5 2 8 6 2 5 1 6 9 1
 2 2 3 3 6 0 1 0 6 5 5 4 4 3 2 2 3 6 9 4 4 9 7 8 8 3 7 3 5 1 7 4 4 8 8 0 8
 8 0 1 2 5 5 2 9 1 6 7 8 5 6 2 3 0 6 7 7 0 6 4 9 0 2 2 4 1 5 9 2 0 5 6 2 4
 8 9 7 8 8 1 8 8 9 2 7 7 7 1 4 1 5 2 7 3 3 6 6 6 1 5 1 1 4 6 3 9 6 2 6 7 2
 6 4 9 7 4 0 6 6 2 0 8 7 

0.843